In [ ]:
#マウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
##作業するディレクトリを指定する.
%cd '/content/drive/MyDrive/Colab_Notebooks/arXiv_data'

/content/drive/MyDrive/Colab_Notebooks/arXiv_data


In [ ]:
#関係するツールの導入
import os
import numpy as np
import pandas as pd
import collections
import re
from string import digits

import sys
import time

In [ ]:
#input_folder_path: 入力となるファイルの保存場所
input_folder_path ='/content/drive/MyDrive/Colab_Notebooks/arXiv_data/Step_6'
#output_folder_path: 出力となるpdfファイルの保存場所
output_folder_path = '/content/drive/MyDrive/Colab_Notebooks/arXiv_data/Step_7'

In [ ]:
#モジュール
module_path = '/content/drive/My Drive/Colab_Notebooks/arXiv_data/module/'
os.chdir(module_path)
import file_processing

In [ ]:
file_names_list = file_processing.read_file_to_list(input_folder_path + "/" + 'k_means_id_list_10_20.txt')

In [ ]:
file_names_list

['0704.2723',
 '0704.3127',
 '0704.3443',
 '0704.3752',
 '0705.0118',
 '0705.1316',
 '0705.2067',
 '0705.2332',
 '0705.2522',
 '0705.3087',
 '0704.1446',
 '0704.1543',
 '0704.1673',
 '0704.1807',
 '0704.1851',
 '0704.1982',
 '0704.1986',
 '0704.2081',
 '0704.2119',
 '0704.2152',
 '0704.1208',
 '0704.1349',
 '0704.1352',
 '0704.1372',
 '0704.1861',
 '0704.1862',
 '0704.1954',
 '0704.2031',
 '0704.2104',
 '0704.2359']

In [ ]:
len(file_names_list)

30

In [ ]:
df = pd.read_csv(input_folder_path + "/df_10_z_value_from_field.csv")
df = df.drop('Unnamed: 0', axis=1)

##クラスタリングの前準備

In [ ]:
#正規化
def minmax_norm(list_input):
    return (list_input - list_input.min()) / ( list_input.max() - list_input.min())

K-平均法

In [ ]:
#k平均法に必要なパッケージ
from matplotlib import pyplot as plt
from sklearn import datasets, preprocessing
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
#クラスタリングの結果からfile名をクラスタごとに分ける
# show_result (str_list,int_array,int) -> str_list_list
def show_result (file, result_k,n):
  result_file = []
  for i in range(n):
    result_file.append([])
  for j in range(len(file)):
    l = result_k[j]
    result_file[l].append(file[j])
  #return print(result_file)
  return result_file

In [ ]:
#傾向となる単語を取りだす
#df: データ k_means_result:k-means法でクラスタリングした結果, n:クラスタリングの個数
def trend (df,k_means_result,n,m,dictionary):
  trend = []
  df['cluster_id']=k_means_result
  for i in range(n):
    trend_n = df[df['cluster_id']==i].mean()
    trend_number = trend_n.drop('cluster_id', axis=0)
    trend_word = trend_number.sort_values(ascending = False)[0:m].index
    trend.append((i,trend_word))
  #return print(trend)
  return trend

階層的クラスタリング

In [ ]:
#階層的クラスタリングに必要なパッケージ
import numpy as np
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import load_iris
from sklearn.cluster import AgglomerativeClustering

In [ ]:
def plot_dendrogram(model, **kwargs):
   # Create linkage matrix and then plot the dendrogram

   # create the counts of samples under each node
    #要素の値が全てゼロの配列を作成する
   counts = np.zeros(model.children_.shape[0])
   n_samples = len(model.labels_)
   for i, merge in enumerate(model.children_):
      current_count = 0
      for child_idx in merge:
         if child_idx < n_samples:
            current_count += 1 # leaf node
         else:
            current_count += counts[child_idx - n_samples]
      counts[i] = current_count

   linkage_matrix = np.column_stack([model.children_, model.distances_,
counts]).astype(float)

   # Plot the corresponding dendrogram
   #dendrogram(linkage_matrix, **kwargs,labels = file_names_list,leaf_rotation=90 )#, orientation='left')
   dendrogram(linkage_matrix, **kwargs,labels = file_names_list, orientation='left')

In [ ]:
df

##データの加工

In [ ]:
#各単語に対して正規化をする．
df_minmax_norm = df.apply(minmax_norm)

##K-means法

In [ ]:
number_cluster = 12

In [ ]:
#k-means
result = KMeans(n_clusters= number_cluster ).fit_predict(df_minmax_norm)

In [ ]:
show_result(file_names_list,result,number_cluster)

[['0704.1851'],
 ['0704.1208', '0704.1861', '0704.2031'],
 ['0704.2723',
  '0704.3752',
  '0705.1316',
  '0705.2332',
  '0705.2522',
  '0705.3087'],
 ['0704.3127',
  '0704.3443',
  '0705.0118',
  '0705.2067',
  '0704.1446',
  '0704.1543',
  '0704.1982',
  '0704.1986',
  '0704.2104',
  '0704.2359'],
 ['0704.1954'],
 ['0704.1349'],
 ['0704.1862'],
 ['0704.1352'],
 ['0704.2152'],
 ['0704.1673', '0704.1807', '0704.2119'],
 ['0704.1372'],
 ['0704.2081']]

In [ ]:
#それぞれのクラスターでよく用いられる単語10個
trend(df,result,number_cluster,10,df_minmax_norm.columns)

[(0, Index(['curvature', 'manifold', 'manifolds', 'riemannian', 'ricci',
         'satisfying', 'bounded', 'compact', 'proved', 'must'],
        dtype='object')),
 (1,
  Index(['anal', 'cauchy', 'solutions', 'solution', 'comm', 'estimate', 'obtain',
         'regularity', 'satisﬁes', 'corollary'],
        dtype='object')),
 (2,
  Index(['algebra', 'algebras', 'associative', 'generated', 'ideal', 'every',
         'commutative', 'multiplication', 'rings', 'assume'],
        dtype='object')),
 (3,
  Index(['algebra', 'vector', 'exists', 'algebras', 'proposition', 'ﬁnitely',
         'remark', 'existence', 'manifold', 'primitive'],
        dtype='object')),
 (4,
  Index(['curvature', 'exists', 'proposition', 'converges', 'holds', 'obtain',
         'estimate', 'sense', 'proved', 'bound'],
        dtype='object')),
 (5,
  Index(['estimates', 'bound', 'estimate', 'obtain', 'supported', 'gradient',
         'inequality', 'prove', 'regularity', 'holds'],
        dtype='object')),
 (6, Index([

##階層的クラスタリング

In [1]:
parm_affinity = 'euclidean'
param_linkage = 'average'
  # setting distance_threshold=0 ensures we compute the full tree.
  # affinity: 距離の出す際の計算方法 
  #linkage : 観測セット間で使用する距離 ward マージされるクラスターの分散を最小限に抑える 
  #distance_threshold クラスターはマージされないリンケージ距離のしきい値
  #n_clusters 検索するクラスターの数
model = AgglomerativeClustering(affinity= parm_affinity,
linkage= param_linkage, 
distance_threshold=0,
n_clusters=None)
model = model.fit(df)
fig = plt.figure( figsize=(6,8))
plt.title(parm_affinity + ', linkage=' + param_linkage )
  # plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode='level')#, p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
#plt.figure(figsize=(6, 8), dpi=300)
  #fig.savefig("dendrogram.pdf")
plt.savefig("dendrogram_z_euclidean_mean.pdf", bbox_inches='tight')
plt.show()

NameError: ignored

In [ ]:
df

,positive,assume,converges,commutative,sense,consequently,ring,primitive,spaces,math,exists,gradient,obtain,mech,problem,belongs,phrases,solution,smooth,associative,proved,vector,must,manifolds,ideal,riemannian,solutions,every,preprint,manifold,proposition,prove,cauchy,compact,comm,ﬁnite,bounded,author,holds,moreover,existence,inﬁnite,uniformly,mathematics,satisfying,estimate,anal,imply,satisﬁes,algebras,inequality,rings,metrics,ricci,denote,remark,corollary,hence,generated,bound,regularity,gives,supported,respect,curvature,algebra,necessarily,multiplication,estimates,enough,ﬁnitely,metric,cluster_id
0,-1.253886,10.965051,-0.452675,-0.562511,0.496078,-0.589093,-0.861419,-0.349321,-0.889574,4.571365,3.919295,-0.552040,1.859003,-0.294534,-1.425202,-0.482759,-0.206739,-1.516170,-0.936770,-0.264600,2.032496,-0.700924,-1.246706,-0.641970,41.090097,-0.445473,-1.256811,37.133097,-0.524027,-0.824744,6.976971,9.271655,-0.382750,-0.982755,16.244653,1.070390,-0.911612,-0.479919,2.299572,1.971077,-0.845709,8.586437,-0.537409,5.260651,-0.700306,-1.020722,-0.395602,-0.557809,7.335347,57.872810,-0.758204,-0.507278,-0.421473,-0.394420,0.376980,-0.892083,1.439636,4.655587,48.485206,-1.099072,-0.351035,-1.208842,0.495854,-1.192471,-0.803605,34.833502,-0.526756,-0.462584,-0.746590,-0.871160,-0.522497,-1.152309,2
1,-1.743478,4.349888,-0.629426,-0.782150,0.751031,-0.819111,-0.362671,32.456671,-1.236917,-0.814331,13.636190,-0.767589,0.816984,-0.409537,-1.981686,0.818602,-0.287462,-1.633458,-1.302541,29.530958,2.958241,2.120925,-1.156320,-0.892633,24.201549,-0.619412,-1.747545,1.817224,-0.728638,-1.146773,4.127856,3.250228,-0.532199,-1.366482,-0.502417,3.487656,-1.267561,2.330042,2.578101,5.739650,20.939707,-1.179320,-0.747246,0.129739,0.053384,-1.419272,-0.550068,4.382162,4.701619,10.641297,-1.054252,-0.705350,-0.586042,-0.548425,2.925383,14.081308,8.748486,17.536232,0.730687,-1.528216,-0.488100,-1.085611,-0.168626,-1.054685,-1.117380,38.554376,-0.732433,4.021282,-1.038104,1.265977,-0.726511,-1.602240,3
2,2.344087,0.531285,-0.488493,4.335679,7.448639,0.937533,3.374527,-0.376961,-0.959963,-0.086321,12.111708,-0.595721,2.737537,-0.317839,-1.537973,-0.520958,-0.223097,-1.636139,-1.010894,-0.285537,1.772886,1.107855,6.091567,-0.692767,0.450594,-0.480721,-1.356257,1.702327,-0.565491,-0.890003,4.864405,0.592660,-0.413036,-1.060517,2.174804,0.757747,-0.983745,3.344207,2.986667,-1.087252,-0.912627,-0.915262,-0.579933,-0.727439,3.214673,-1.101488,-0.426904,-0.601946,-0.922533,46.969257,5.293988,1.279502,-0.454823,-0.425629,3.903298,2.154504,6.546106,-0.325117,-1.090499,1.344430,-0.378811,0.996394,-0.844362,1.823098,-0.867191,54.879485,1.190943,1.504218,-0.805665,0.123908,-0.563841,-1.243487,3
3,-0.177122,7.982153,-0.658256,11.408649,-0.258505,0.310874,4.337019,-0.507964,1.026209,1.089501,1.792578,-0.802748,-1.709813,-0.428296,-2.072455,-0.702004,3.025779,-2.204736,-1.362203,2.214266,1.790044,2.340220,7.569325,-0.933519,5.417916,-0.647783,-1.827589,5.513690,-0.762013,-1.199300,-1.408666,6.852398,-0.556576,-1.429072,-0.525430,0.903109,-1.325620,-0.697874,0.128894,3.314388,-0.416440,-1.233338,-0.781473,2.080700,-1.018349,-1.484281,-0.575263,0.421832,1.170718,46.414801,-1.102541,12.819945,-0.612885,-0.573545,3.198449,1.016040,3.498114,4.415064,18.272342,-1.598214,3.407769,-1.188671,2.378496,0.573851,-1.168561,53.093759,-0.765981,5.274256,-1.085653,-1.266796,4.505323,-1.675629,2
4,-1.199651,-1.318553,-0.433095,1.320131,-0.748524,-0.563613,6.457825,-0.334211,-0.851097,-1.293071,0.649039,-0.528162,5.578490,-0.281794,-1.363557,-0.461878,-0.197797,-1.450590,-0.896252,-0.253155,-0.693963,-1.349409,-1.192781,-0.614202,-0.709150,-0.426204,-1.202449,-1.172869,-0.501361,-0.789070,10.565301,-0.036431,-0.366195,4.379245,-0.345703,-0.788472,1.421565,-0.459161,0.235802,1.111561,-0.809129,-0.811465,-0.514165,4.007349,2.315488,-0.976572,-0.378490,-0.533681,1.627948,5.323126,-0.725409,28.363140,-0.403243,-0.377360,3.036513,9.694205,-0.829698,0.016174,-0.966829,-1.051534,-0.3358

In [ ]:
pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/arXiv_data/Step_4'+ "/0704.1982.csv")

,Unnamed: 0,__init__,bin,caching,call,disable_caching,dist,doc,exit,extract_text,extract_text_to_fp,file,get_pages,high_level,last,lib,line,local,locals,main,module,most,not,object,outfp,packages,parser,password,pdf,pdfdocument,pdfminer,pdfpage,pdfparser,pdfsyntaxerror,pdftxt,python,raise,really,recent,root,sys,this,traceback,usr,vars
0,0,1,3,3,1,1,3,1,1,2,2,6,1,2,1,3,6,6,1,2,1,1,1,2,1,3,1,2,2,2,5,1,1,2,3,3,1,2,1,2,1,2,1,6,1


In [ ]:
pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/arXiv_data/Step_4'+ "/0705.2067.csv")

,Unnamed: 0,__init__,bin,caching,call,disable_caching,dist,doc,exit,extract_text,extract_text_to_fp,file,get_pages,high_level,last,lib,line,local,locals,main,module,most,not,object,outfp,packages,parser,password,pdf,pdfdocument,pdfminer,pdfpage,pdfparser,pdfsyntaxerror,pdftxt,python,raise,really,recent,root,sys,this,traceback,usr,vars
0,0,1,3,3,1,1,3,1,1,2,2,6,1,2,1,3,6,6,1,2,1,1,1,2,1,3,1,2,2,2,5,1,1,2,3,3,1,2,1,2,1,2,1,6,1
